# E13 - Exercise 1: Graph Algorithms
In the directory E13-1-GDS-Examples at the repository soft2022spring-DS you will find
airports data and processing graph algorithms.\
Your task is to wrangle the available data into a graph database and create a Python
application, which performs the major graph data science algorithms on the database.
Reuse the available code as much as possible.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neo4j import GraphDatabase

### Make a connection class

In [2]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

### Connect to the Neo4J DB.

In [3]:
uri = 'bolt://localhost:7687'
pwd = 'test1234'

conn = Neo4jConnection(uri=uri, user="neo4j", pwd=pwd)

### Return the number of nodes.

In [4]:
result = conn.query('MATCH (n) RETURN COUNT(n) AS count')

print('Number of nodes: ', result[0]['count'])

Number of nodes:  8627


### Return the number of relationships.

In [5]:
result = conn.query('MATCH ()-[r]->() RETURN COUNT(r) AS count')

print('Number of relationships: ', result[0]['count'])

Number of relationships:  73954


### Return the amount of 'In Degree' relations for each city (Limit by 10)

In [6]:
in_degree_query = """MATCH (a:Airport)
                     WITH a, SIZE(()-[:HAS_ROUTE]->(a)) AS in_degree
                     RETURN a.city as City, in_degree as InDegree
                     ORDER BY in_degree DESC
                     LIMIT 10
                  """

in_degree_df = pd.DataFrame([dict(_) for _ in conn.query(in_degree_query)])
in_degree_df.head(10)

,City,InDegree
0,Frankfurt,303
1,Paris,291
2,Amsterdam,280
3,Istanbul,268
4,Munich,265
5,Chicago,257
6,Dallas,248
7,Atlanta,242
8,Dubai,237
9,London,226


### Return the amount of 'Total Degree' relations for each city (Limit by 10)

In [7]:
total_degree_query = """MATCH (a:Airport)
                        WITH a, SIZE(()-[:HAS_ROUTE]-(a)) AS total_degree
                        RETURN a.city as City, total_degree as TotalDegree
                        ORDER BY total_degree DESC
                        LIMIT 10
                     """

total_degree_df = pd.DataFrame([dict(_) for _ in conn.query(total_degree_query)])
total_degree_df.head(10)

,City,TotalDegree
0,Frankfurt,610
1,Paris,584
2,Istanbul,575
3,Amsterdam,562
4,Munich,535
5,Chicago,521
6,Dallas,499
7,Dubai,484
8,Atlanta,484
9,Beijing,469


### Return how many airports are exactly 1, exactly 2, and then 1 or 2 hops from a target airport.

In [8]:
one_hop_query = "MATCH (a:Airport {iata: 'DEN'})-[:HAS_ROUTE]->(a2) RETURN COUNT(DISTINCT a2)"
two_hop_query = "MATCH (a:Airport {iata: 'DEN'})-[:HAS_ROUTE*2]->(a2) RETURN COUNT(DISTINCT a2)"
one_or_two_query = "MATCH (a:Airport {iata: 'DEN'})-[:HAS_ROUTE*1..2]->(a2) RETURN COUNT(DISTINCT a2)"

print('Number of airports within exactly 1 hop: ', conn.query(one_hop_query))
print('Number of airports within exactly 2 hops: ', conn.query(two_hop_query))
print('Number of airports within 1 or 2 hops: ', conn.query(one_or_two_query))

Number of airports within exactly 1 hop:  [<Record COUNT(DISTINCT a2)=216>]
Number of airports within exactly 2 hops:  [<Record COUNT(DISTINCT a2)=1218>]
Number of airports within 1 or 2 hops:  [<Record COUNT(DISTINCT a2)=1231>]


***

### Creating a graph projection

In [9]:
projection_query = """CALL gds.graph.create(
                      'routes',
                      'Airport',
                      'HAS_ROUTE'
                      )
                      YIELD
                          graphName, nodeProjection, nodeCount, relationshipProjection, relationshipCount
                    """

conn.query(projection_query)

Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.create`: Caused by: java.lang.IllegalArgumentException: A graph with name 'routes' already exists.}


***
### Page Rank
Score based on the weights of each airports connections to other airports.

The PageRank algorithm measures the importance of each node within the graph, based on the number incoming relationships and the importance of the corresponding source nodes. The underlying assumption roughly speaking is that a page is only as important as the pages that link to it.

In [17]:

pagerank_query = """CALL gds.pageRank.stream('routes')
                    YIELD nodeId, score
                    RETURN gds.util.asNode(nodeId).iata AS Iata, 
                        gds.util.asNode(nodeId).descr AS Description, score as Score
                    ORDER BY score DESC, Iata ASC
                    """

pr_df = pd.DataFrame([dict(_) for _ in conn.query(pagerank_query)])
pr_df.head(10)

,Iata,Description,Score
0,DFW,Dallas/Fort Worth International Airport,11.979783
1,ORD,Chicago O'Hare International Airport,11.162988
2,DEN,Denver International Airport,10.997299
3,ATL,Hartsfield - Jackson Atlanta International Air...,10.389948
4,IST,Istanbul International Airport,8.425801
5,CDG,Paris Charles de Gaulle,8.401469
6,IAH,George Bush Intercontinental,8.341141
7,FRA,Frankfurt am Main,8.203205
8,LAX,Los Angeles International Airport,8.193558
9,CLT,Charlotte Douglas International Airport,7.873303


***
### Show statistics on the routes
The write execution mode extends the stats mode with an important side effect: writing the score for each node as a property to the Neo4j database. The name of the new property is specified using the mandatory configuration parameter writeProperty. The result is a single summary row, similar to stats, but with some additional metrics. The write mode enables directly persisting the results to the database.

In [11]:
# Page Rank is an algorithm that measures the transitive influence or connectivity of nodes.

pagerank_write_query = """CALL gds.pageRank.write('routes', 
                            {
                                writeProperty: 'pagerank'
                            }
                          )
                          YIELD nodePropertiesWritten, ranIterations
                          """

conn.query(pagerank_write_query)

[<Record nodePropertiesWritten=3503 ranIterations=20>]

***
### Path finding (Dijkstra's algorithm for shortest path)

In [12]:
weighted_proj_query = """CALL gds.graph.create(
                              'routes-weighted',
                              'Airport',
                              'HAS_ROUTE',
                                  {
                                      relationshipProperties: 'distance'
                                  }
                          )
                      """

conn.query(weighted_proj_query)

Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.create`: Caused by: java.lang.IllegalArgumentException: A graph with name 'routes-weighted' already exists.}


In [13]:
sp_query = """MATCH (source:Airport {iata: 'DEN'}), (target:Airport {iata: 'MLE'})
              CALL gds.shortestPath.dijkstra.stream('routes-weighted', {
                  sourceNode: source,
                  targetNode: target,
                  relationshipWeightProperty: 'distance'
              })
              YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
              RETURN
                  index,
                  gds.util.asNode(sourceNode).iata AS SourceNodeName,
                  gds.util.asNode(targetNode).iata AS TargetNodeName,
                  totalCost,
                  [nodeId IN nodeIds | gds.util.asNode(nodeId).iata] AS nodeNames,
                  costs,
                  nodes(path) as path
              ORDER BY index
              """

sp_df = pd.DataFrame([dict(_) for _ in conn.query(sp_query)])
sp_df.head(10)

,index,SourceNodeName,TargetNodeName,totalCost,nodeNames,costs,path
0,0,DEN,MLE,9704.0,"[DEN, KEF, HEL, VKO, MLE]","[0.0, 3556.0, 5074.0, 5629.0, 9704.0]","[(descr, altitude, longest, iata, city, icao, ..."


***
### Clustering (Louvian)

In [14]:
louvain_query = """CALL gds.louvain.stream('routes')
                   YIELD nodeId, communityId
                   RETURN 
                       communityId,
                       SIZE(COLLECT(gds.util.asNode(nodeId).iata)) AS number_of_airports,
                       COLLECT(gds.util.asNode(nodeId).city) AS city
                   ORDER BY number_of_airports DESC, communityId
                   """

louvain_df = pd.DataFrame([dict(_) for _ in conn.query(louvain_query)])
louvain_df.head()

,communityId,number_of_airports,city
0,3018,684,"[Atlanta, Anchorage, Austin, Nashville, Boston..."
1,3307,498,"[London, London, Paris, Frankfurt, Helsinki, D..."
2,2784,418,"[Dubai, New Delhi, Mumbai, Doha, Calicut, Hyde..."
3,2838,250,"[Tokyo, Singapore, Hong Kong, Beijing, Shangha..."
4,2927,187,"[Sydney, Melbourne, Perth, Auckland, Wellingto..."


***
### Node Similarity

In [16]:
ns_query = """CALL gds.nodeSimilarity.stream(
                   'routes',
                   {
                       topK: 3
                   }
               )
               YIELD node1, node2, similarity
               RETURN 
                   gds.util.asNode(node1).city AS City1, 
                   COLLECT(gds.util.asNode(node2).city) AS City2, 
                   COLLECT(similarity) AS similarity
               ORDER BY similarity[0] DESC
               """

ns_df = pd.DataFrame([dict(_) for _ in conn.query(ns_query)])
ns_df.head(10)

,City1,City2,similarity
0,Santa Fe,"[Flagstaff, Durango, Yuma]","[1.0, 1.0, 0.75]"
1,Manhattan,"[Sioux City, Champaign/Urbana, Columbia]","[1.0, 0.6666666666666666, 0.6666666666666666]"
2,Monroe,"[Fort Hood/Killeen, Alexandria, Fort Smith]","[1.0, 1.0, 0.6666666666666666]"
3,San Angelo,"[Abilene, Waco, Texarkana]","[1.0, 1.0, 1.0]"
4,Wichita Falls,"[Abilene, Waco, Texarkana]","[1.0, 1.0, 1.0]"
5,Tyler,"[Lake Charles, College Station, Brownsville]","[1.0, 1.0, 0.6666666666666666]"
6,Burlington,"[Quincy, Decatur, Cape Girardeau, Greensboro, ...","[1.0, 1.0, 0.6666666666666666, 0.6666666666666..."
7,Florence,"[Greenville, Lynchburg, Staunton/Waynesboro/Ha...","[1.0, 1.0, 0.5]"
8,Topeka,"[Hancock, Dubuque, Waterloo]","[1.0, 1.0, 1.0]"
9,Marquette,"[La Crosse, Mosinee, Saginaw]","[1.0, 1.0, 0.75]"
